## 试着统计一下每口井的各个大层（砂组）的厚度


In [ ]:
import numpy as np
import pandas as pd

# 读取处理后的数据
file_well = "../data/well_horizon_processed.xlsx"
data_well = pd.read_excel(file_well)

print("数据形状：", data_well.shape)
print("数据前5行：")
print(data_well.head())

# 定义列名
well_column = "Well"
surface_column = "Surface"
md_column = "MD"

# 定义砂组层序（从上到下）
sand_groups = [
    "H1-1",
    "H2-1",
    "H3Ding(23D)",
    "H4ding(34D)",
    "H5ding",
    "H6ding(53D)",
    "H7ding(63D)",
    "H8ding(72D)",
    "P0(H83D)",
]

print(f"\n砂组层序（从上到下）：")
for i, sg in enumerate(sand_groups, 1):
    print(f"  {i}. {sg}")

# 筛选出包含这些层位的数据
data_filtered = data_well[data_well[surface_column].isin(sand_groups)].copy()
print(f"\n筛选后数据: {len(data_filtered)} 行")

# 获取所有井名
wells = sorted(data_filtered[well_column].unique())
print(f"共有 {len(wells)} 口井")

# 初始化结果列表
results = []

# 对每口井进行处理
for well in wells:
    well_data = data_filtered[data_filtered[well_column] == well].copy()

    # 创建一个字典存储每个层位的MD值
    surface_md = {}
    for _, row in well_data.iterrows():
        surface_md[row[surface_column]] = row[md_column]

    # 计算相邻层位之间的厚度
    result_row = {"Well": well}

    for i in range(len(sand_groups) - 1):
        upper_surface = sand_groups[i]
        lower_surface = sand_groups[i + 1]

        # 层厚名称：H1表示H1-1到H2-1之间的厚度
        thickness_name = f"H{i + 1}层厚"

        # 如果两个层位都存在，计算厚度
        if upper_surface in surface_md and lower_surface in surface_md:
            thickness = surface_md[lower_surface] - surface_md[upper_surface]
            result_row[thickness_name] = thickness
        else:
            result_row[thickness_name] = np.nan

    results.append(result_row)

# 创建结果DataFrame
df_thickness = pd.DataFrame(results)

# 显示统计信息
print(f"\n=== 砂组厚度统计结果 ===")
print(f"结果数据形状: {df_thickness.shape}")
print(f"\n前5口井的统计结果:")
print(df_thickness.head())

# 统计每个层厚的基本信息
print(f"\n各层厚度统计:")
thickness_columns = [col for col in df_thickness.columns if "层厚" in col]
for col in thickness_columns:
    valid_count = df_thickness[col].notna().sum()
    if valid_count > 0:
        stats = df_thickness[col].describe()
        print(f"\n{col}:")
        print(f"  有效数据: {valid_count}/{len(df_thickness)} 口井 ({valid_count / len(df_thickness) * 100:.1f}%)")
        print(f"  平均值: {stats['mean']:.2f} m")
        print(f"  中位数: {stats['50%']:.2f} m")
        print(f"  最小值: {stats['min']:.2f} m")
        print(f"  最大值: {stats['max']:.2f} m")
        print(f"  标准差: {stats['std']:.2f} m")
    else:
        print(f"\n{col}: 无有效数据")

# 统计缺失情况
print(f"\n=== 数据缺失统计 ===")
for col in thickness_columns:
    missing_count = df_thickness[col].isna().sum()
    if missing_count > 0:
        missing_wells = df_thickness[df_thickness[col].isna()]["Well"].tolist()
        print(f"{col}: 缺失 {missing_count} 口井 - {missing_wells}")

# 保存结果
output_file = "../data/sand_group_thickness.xlsx"
df_thickness.to_excel(output_file, index=False)
print(f"\n结果已保存到: {output_file}")

# 显示完整结果
print(f"\n=== 完整结果预览 ===")
print(df_thickness.to_string())

In [ ]:
# 显示完整结果
print(f"\n=== 完整结果预览 ===")
print(df_thickness.to_string())

# 删除存在NaN层厚的井
print(f"\n=== 删除存在缺失值的井 ===")
print(f"删除前井数: {len(df_thickness)}")

# 找出所有层厚列
thickness_columns = [col for col in df_thickness.columns if "层厚" in col]

# 删除任何层厚列存在NaN的井
df_thickness_complete = df_thickness.dropna(subset=thickness_columns).copy()

removed_count = len(df_thickness) - len(df_thickness_complete)
print(f"删除了 {removed_count} 口井（存在缺失层厚数据）")
print(f"删除后井数: {len(df_thickness_complete)}")

if removed_count > 0:
    removed_wells = set(df_thickness["Well"]) - set(df_thickness_complete["Well"])
    print(f"被删除的井: {sorted(removed_wells)}")

# 计算平均层厚
if len(df_thickness_complete) > 0:
    print(f"\n=== 完整数据井的平均层厚统计 ===")
    print(f"基于 {len(df_thickness_complete)} 口完整数据的井")

    # 计算每个层厚的平均值
    avg_thickness = {}
    for col in thickness_columns:
        avg_val = df_thickness_complete[col].mean()
        std_val = df_thickness_complete[col].std()
        min_val = df_thickness_complete[col].min()
        max_val = df_thickness_complete[col].max()

        avg_thickness[col] = avg_val

        print(f"\n{col}:")
        print(f"  平均值: {avg_val:.2f} m")
        print(f"  标准差: {std_val:.2f} m")
        print(f"  最小值: {min_val:.2f} m")
        print(f"  最大值: {max_val:.2f} m")

    # 计算总平均层厚
    total_avg = sum(avg_thickness.values())
    print(f"\n所有层厚平均值之和: {total_avg:.2f} m")
    print(f"单层平均厚度: {total_avg / len(thickness_columns):.2f} m")

    # 创建平均值汇总表
    avg_summary = pd.DataFrame(
        {
            "层位": thickness_columns,
            "平均厚度(m)": [avg_thickness[col] for col in thickness_columns],
            "标准差(m)": [df_thickness_complete[col].std() for col in thickness_columns],
            "最小值(m)": [df_thickness_complete[col].min() for col in thickness_columns],
            "最大值(m)": [df_thickness_complete[col].max() for col in thickness_columns],
        }
    )

    print(f"\n平均厚度汇总表:")
    print(avg_summary.to_string(index=False))

    # 保存完整数据
    output_file_complete = "../data/sand_group_thickness_complete.xlsx"

    # 使用ExcelWriter保存多个sheet
    with pd.ExcelWriter(output_file_complete) as writer:
        df_thickness_complete.to_excel(writer, sheet_name="完整数据", index=False)
        avg_summary.to_excel(writer, sheet_name="平均厚度统计", index=False)

    print(f"\n完整数据已保存到: {output_file_complete}")
    print(f"  - Sheet '完整数据': {len(df_thickness_complete)} 口井的层厚数据")
    print(f"  - Sheet '平均厚度统计': 各层平均厚度统计")

    # 显示完整数据预览
    print(f"\n=== 完整数据预览 ===")
    print(df_thickness_complete.to_string())
else:
    print("\n警告: 没有井具有完整的层厚数据！")